# Groupby

Como introducción al groupby (sí, eso "agrupar por") vamos a plantear unas cuestiones a nuestros datos de viajes aéreos que ya planteamos en sesiones anteriores. Recordaremos como las resolvimos y eso nos dará pie a ver una forma más eficiente de hacerlo a través de las agrupaciones con `groupby` y de ahí completaremos con más detalles y posibilidades adicionales.

Por tanto, como en otras sesiones, comencemos creando nuestro DataFrame a partir de los datos de vuelos:




In [4]:
import pandas as pd

In [19]:
df_aviones = pd.read_csv("C:/Users/david/Downloads/dataset_viajes.csv", index_col = "Id_vuelo")
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN,51.0
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.13254400001,1167.0
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN,626.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.01,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744,518.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,"62300,238",391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.5664,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868,818.0


# ¿Para qué groupby?

En sesiones anteriores le "preguntamos" a nuestro DataFrame, `df_aviones`, por la media de consumo de los aviones por compañía, o algo parecido… y lo hacíamos de esta forma, válida, pero farragosa:


In [11]:
df_aviones["consumo_kg"] = pd.to_numeric(df_aviones["consumo_kg"], errors='coerce')

In [12]:

for compania in df_aviones["Aircompany"].unique():
    es_compania = df_aviones["Aircompany"] == compania
    print(f"Para la compañía {compania}:")
    for avion in df_aviones["avion"].unique():
        es_avion = df_aviones["avion"] == avion
        consumo = df_aviones.loc[es_compania & es_avion, "consumo_kg"].mean()
        print(f"Tipo <{avion}> consumo medio por vuelo <{consumo:.2f}>")

Para la compañía Airnar:
Tipo <Boeing 737> consumo medio por vuelo <25915.73>
Tipo <Airbus A380> consumo medio por vuelo <99714.04>
Tipo <Boeing 747> consumo medio por vuelo <84408.88>
Tipo <Airbus A320> consumo medio por vuelo <15795.91>
Para la compañía FlyQ:
Tipo <Boeing 737> consumo medio por vuelo <19675.47>
Tipo <Airbus A380> consumo medio por vuelo <118307.15>
Tipo <Boeing 747> consumo medio por vuelo <77214.70>
Tipo <Airbus A320> consumo medio por vuelo <12581.06>
Para la compañía TabarAir:
Tipo <Boeing 737> consumo medio por vuelo <12926.55>
Tipo <Airbus A380> consumo medio por vuelo <64832.85>
Tipo <Boeing 747> consumo medio por vuelo <58602.56>
Tipo <Airbus A320> consumo medio por vuelo <16210.58>
Para la compañía MoldaviAir:
Tipo <Boeing 737> consumo medio por vuelo <30610.04>
Tipo <Airbus A380> consumo medio por vuelo <146952.81>
Tipo <Boeing 747> consumo medio por vuelo <134801.77>
Tipo <Airbus A320> consumo medio por vuelo <7754.04>
Para la compañía PamPangea:
Tipo <Boei

Esta forma, aunque eficaz en términos humanos (es decir, una persona obtiene la información que quería), no es muy buena en términos de programación por dos motivos:

1. **El código es farragoso**: Incluso si se convierte en una función con las columnas como parámetros, sería complejo de mantener.

2. **La salida no es muy manejable posteriormente**: Aunque es cierto que podríamos crear una estructura de salida, esto complicaría aún más el código.

Es para este tipo de situaciones para lo que aparece el método `groupby`, pero ojo, no solo para estas, como iremos viendo en esta y las siguientes sesiones.

In [15]:
df_aviones.groupby(["Aircompany", "avion"])["consumo_kg"].mean()

Aircompany  avion      
Airnar      Airbus A320     15795.910610
            Airbus A380     99714.041295
            Boeing 737      25915.730409
            Boeing 747      84408.875738
FlyQ        Airbus A320     12581.060246
            Airbus A380    118307.145259
            Boeing 737      19675.467307
            Boeing 747      77214.695841
MoldaviAir  Airbus A320      7754.038472
            Airbus A380    146952.811915
            Boeing 737      30610.038293
            Boeing 747     134801.766454
PamPangea   Airbus A320      8160.900592
            Airbus A380    126474.081366
            Boeing 737      25693.230962
            Boeing 747     133869.772874
TabarAir    Airbus A320     16210.582844
            Airbus A380     64832.847670
            Boeing 737      12926.549025
            Boeing 747      58602.560652
Name: consumo_kg, dtype: float64

In [16]:
agrupacion = df_aviones.groupby(["Aircompany", "avion"])["consumo_kg"].mean()

In [17]:
print(type(agrupacion))

<class 'pandas.core.series.Series'>


Nos ha devuelto una serie, por lo tanto podemos manipularla como serie

In [18]:
agrupacion.index

MultiIndex([(    'Airnar', 'Airbus A320'),
            (    'Airnar', 'Airbus A380'),
            (    'Airnar',  'Boeing 737'),
            (    'Airnar',  'Boeing 747'),
            (      'FlyQ', 'Airbus A320'),
            (      'FlyQ', 'Airbus A380'),
            (      'FlyQ',  'Boeing 737'),
            (      'FlyQ',  'Boeing 747'),
            ('MoldaviAir', 'Airbus A320'),
            ('MoldaviAir', 'Airbus A380'),
            ('MoldaviAir',  'Boeing 737'),
            ('MoldaviAir',  'Boeing 747'),
            ( 'PamPangea', 'Airbus A320'),
            ( 'PamPangea', 'Airbus A380'),
            ( 'PamPangea',  'Boeing 737'),
            ( 'PamPangea',  'Boeing 747'),
            (  'TabarAir', 'Airbus A320'),
            (  'TabarAir', 'Airbus A380'),
            (  'TabarAir',  'Boeing 737'),
            (  'TabarAir',  'Boeing 747')],
           names=['Aircompany', 'avion'])

El índice es un índice multidimensional (cada valor es una tupla) en los que no vamos a profundizar, pero que se puede operar como cualquier otro índice de una serie:

In [21]:
print(agrupacion[("Airnar","Airbus A320")])


15795.910610434781


# Groupby con cierto detalle

El método `groupby` de un DataFrame (también existe para Series, pero puedes consultarlo [aquí](enlace)) tiene varios argumentos interesantes. El primero, que ya hemos visto, es la lista de columnas por las que queremos agrupar (puede ser solo una). Otro argumento importante es `as_index`, que veremos más adelante. 

Antes de continuar, es interesante destacar que `groupby` es un método "vago" [sí, como Jose Mota, "si hay que ir se va, pero ir pa ná"]. Veámoslo ejecutando solo el `groupby`:

In [22]:
df_aviones.groupby(["Aircompany","avion"])

Nos ha devuelto un objeto `groupby`, lo cual, de primeras, es lo mismo que nada. Para obtener algo útil, hay que especificar qué operación debe realizarse con las columnas restantes:

In [24]:
df_aviones.groupby(["Aircompany","avion"]).count()

Origen  Destino  Distancia  consumo_kg  duracion
Aircompany avion                                                        
Airnar     Airbus A320      29       29         24          26        28
           Airbus A380      55       55         47          42        48
           Boeing 737       40       40         32          29        33
           Boeing 747       56       56         48          50        49
FlyQ       Airbus A320      27       27         20          26        21
           Airbus A380      48       48         44          43        44
           Boeing 737       44       44         41          39        34
           Boeing 747       54       54         51          47        47
MoldaviAir Airbus A320      31       31         27          29        24
           Airbus A380      69       69         61          58        60
           Boeing 737       68       68         60          60        62
           Boeing 747       58       58         49          53        44
PamPangea  Airbus A320      26       26         23          25        20
           Airbus A380      58       58         49          48        51
           Boeing 737       61       61         52          55        50
           Boeing 747       47       47         38          39        42
TabarAir   Airbus A320      49       49         46          42        42
           Airbus A380      60       60         56          51        50
           Boeing 737       55       55         46          48        46
           Boeing 747       65       65         58          52        58

In [25]:
df_aviones.groupby(["Aircompany","Destino"]).count()

Origen  Distancia  avion  consumo_kg  duracion
Aircompany Destino                                                    
Airnar     Bali             32         26     32          26        27
           Cincinnati       36         32     36          31        32
           Cádiz            23         22     23          18        20
           GINEbRA           1          1      1           0         1
           GINEbRa           1          1      1           1         1
...                        ...        ...    ...         ...       ...
TabarAir   Ginebra          29         26     29          26        23
           Londres          41         38     41          33        39
           Los Angeles      34         29     34          29        29
           Nueva York       44         37     44          37        38
           Roma             44         41     44          38        38

[79 rows x 5 columns]

Pero puede que queramos ver, por ejemplo, la duración media del vuelo y el consumo medio realizado por Compañía, Origen y Destino:

In [38]:
df_aviones.groupby(["Aircompany","Origen","Destino"]).mean()

TypeError: agg function failed [how->mean,dtype->object]

Nos da error porque si la función de agregación no se puede aplicar a alguna de las columnas que no están involucradas en la agrupación y no se han filtrado, en este caso `"avion"`, que es un `str`, no admite la media, lo que genera un error (contar filas no da error :-).

Tendríamos que filtrar previamente, como hemos hecho con `"consumo_kg"`, y quedarnos con las columnas que queremos o, al menos, con aquellas para las que la función final que aplicamos (`mean`, en este caso) sea válida.

In [39]:
df_medias = df_aviones.groupby(["Aircompany","Origen","Destino"])[["consumo_kg","duracion"]].mean()

In [36]:
df_aviones['consumo_kg'] = pd.to_numeric(df_aviones['consumo_kg'], errors='coerce')

In [30]:
df_aviones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Air_PaGi_10737 to Air_PaCi_10737
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Aircompany  1000 non-null   object 
 1   Origen      1000 non-null   object 
 2   Destino     1000 non-null   object 
 3   Distancia   872 non-null    float64
 4   avion       1000 non-null   object 
 5   consumo_kg  862 non-null    object 
 6   duracion    853 non-null    float64
dtypes: float64(2), object(5)
memory usage: 62.5+ KB


In [40]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion
Id_vuelo,,,,,,,
Air_PaGi_10737,Airnar,París,Ginebra,411.0,Boeing 737,NaN,51.0
Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,33479.132544,1167.0
Tab_GiLo_11380,TabarAir,Ginebra,Los Angeles,9103.0,Airbus A380,NaN,626.0
Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,17027.010000,503.0
Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,86115.744000,518.0
...,...,...,...,...,...,...,...
Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,NaN,391.0
Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,194854.566400,1326.0
Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,128983.868000,818.0


In [42]:
### Resultados para la compañia Airnar

df_medias.loc["Airnar"]

consumo_kg     duracion
Origen      Destino                                
Bali        Cincinnati   127476.462235  1270.000000
            Cádiz         89228.705436  1112.000000
            GInEBrA      138602.919000   845.000000
            Ginebra      145212.964400   845.000000
            Los Angeles  153932.767333   876.000000
            París        119152.121600   902.400000
Cincinnati  Bali          39073.873176  1293.333333
            Cádiz         17384.025600   461.000000
            Ginebra       59738.546760   519.000000
            Los Angeles   27524.246400   231.600000
            París         48155.492840   491.000000
Cádiz       Bali          74406.889440  1131.666667
            Cincinnati    47950.208640   495.750000
            Ginebra       11764.845000   140.000000
            Los Angeles   68524.203384   695.666667
            París         13368.230083   119.200000
Ginebra     Bali         144513.015325   845.000000
            Cincinnati    77237.825229   501.500000
            Cádiz         11515.927500   140.333333
            Los Angeles   65199.266513   716.666667
            París          3545.525750    47.666667
Los Angeles Bali         110210.887827  1078.125000
            Cincinnati    28004.709950   233.000000
            Cádiz         63072.791600   687.500000
            GInEBRa                NaN   626.000000
            GinEbra       22783.898700          NaN
            Ginebra       65410.152680   728.000000
            París         60326.824950   668.500000
París       Bali          85230.104680   878.285714
            Cincinnati    26481.555100   517.500000
            Cádiz         15125.129250   117.600000
            GINEbRA                NaN    47.000000
            GINEbRa                NaN    51.000000
            Ginebra        2875.232700    49.666667
            Los Angeles   23168.820096   745.000000

In [ ]:
##Resultado para vuelos Paris-Cadiz

In [46]:
df_medias.loc[:,"París","Cádiz"]

,consumo_kg,duracion
Aircompany,,
Airnar,15125.12925,117.6
MoldaviAir,3711.98910,125.0


In [ ]:
##Resultado para vuelos Ginebra  y Nueva York de Fly Q

In [47]:
df_medias.loc["FlyQ",:,["Ginebra","Nueva York"]]

consumo_kg     duracion
Aircompany Origen     Destino                               
FlyQ       Bali       Ginebra      89401.421270   845.000000
                      Nueva York  205591.856496  1305.000000
           Barcelona  Ginebra       3048.309000    67.000000
                      Nueva York   49245.632880   478.000000
           Cincinnati Ginebra      31489.354022   563.166667
                      Nueva York    6576.099050    84.125000
           Ginebra    Nueva York   52479.317869   460.200000
           Nueva York Ginebra      61314.038800   447.500000
           Roma       Ginebra       1751.980000    68.000000
                      Nueva York   49384.456793   558.666667

Nosotros, en general, si queremos conservar la salida de un `groupby` para seguir procesándola, emplearemos el argumento `as_index` con valor `False`.

In [49]:
resultado_no_index= df_aviones.groupby(["Aircompany","Origen","Destino"],as_index= False)[["consumo_kg","duracion"]].mean()

In [50]:
resultado_no_index

,Aircompany,Origen,Destino,consumo_kg,duracion
0,Airnar,Bali,Cincinnati,127476.462235,1270.000000
1,Airnar,Bali,Cádiz,89228.705436,1112.000000
2,Airnar,Bali,GInEBrA,138602.919000,845.000000
3,Airnar,Bali,Ginebra,145212.964400,845.000000
4,Airnar,Bali,Los Angeles,153932.767333,876.000000
...,...,...,...,...,...
196,TabarAir,Roma,Cincinnati,59852.929714,568.571429
197,TabarAir,Roma,Ginebra,6032.953600,67.333333
198,TabarAir,Roma,Londres,11199.447729,119.857143
199,TabarAir,Roma,Los Angeles,26061.681558,764.500000


# Groupby: Funciones de Agregación Avanzadas

Hasta ahora hemos visto funciones de agregación sencillas (la función que se pone al final del `groupby` para que "haga algo" con las columnas indicadas). En esta sesión, vamos a aumentar esa capacidad. ¿Cómo? Pues aplicando funciones por columna, es decir, no siempre la misma función a cada columna, y además permitiendo que sea una función personalizada (al estilo `apply`, del que tanto te acuerdas).

Y, ¿cómo no?, comencemos creando nuestro DataFrame a partir de los datos de vuelos:

In [51]:
df_aviones = pd.read_csv("C:/Users/david/Downloads/dataset_inicial_aviones.csv")


# Agg: Funciones de usuario

Supongamos que ahora queremos obtener, por Compañía y tipo de Avión, el número de vuelos, el consumo medio y el destino más visitado. ¡Vaya tarea!  
Descompongámoslo en tres pequeños problemas y luego veremos cómo juntarlos con la ayuda del método **agg**.

1. **Por compañía y tipo de avión, el número de vuelos**: Este es fácil, lo hicimos en la píldora anterior:

In [52]:
df_aviones.groupby(["Aircompany","avion"])["Origen"].count()

Aircompany  avion      
Airnar      Airbus A320    37
            Airbus A380    64
            Boeing 737     49
            Boeing 747     68
FlyQ        Airbus A320    34
            Airbus A380    61
            Boeing 737     54
            Boeing 747     67
MoldaviAir  Airbus A320    35
            Airbus A380    74
            Boeing 737     84
            Boeing 747     71
PamPangea   Airbus A320    31
            Airbus A380    73
            Boeing 737     66
            Boeing 747     61
TabarAir    Airbus A320    61
            Airbus A380    71
            Boeing 737     62
            Boeing 747     77
Name: Origen, dtype: int64

In [53]:
df_grouped = df_aviones.groupby(["Aircompany","avion"])

2. Por compañia y tipo de avion, el cosumo medio.

In [54]:
df_grouped ["consumo_kg"].mean()

Aircompany  avion      
Airnar      Airbus A320     15233.354249
            Airbus A380    101402.622197
            Boeing 737      25213.516883
            Boeing 747      87208.773559
FlyQ        Airbus A320     12255.748389
            Airbus A380    116860.268168
            Boeing 737      20474.087328
            Boeing 747      86157.201917
MoldaviAir  Airbus A320      8422.846644
            Airbus A380    149316.553108
            Boeing 737      30233.624810
            Boeing 747     133625.459193
PamPangea   Airbus A320      7689.835407
            Airbus A380    135216.291382
            Boeing 737      28712.313147
            Boeing 747     127566.722008
TabarAir    Airbus A320     15545.639162
            Airbus A380     62575.392039
            Boeing 737      12305.056732
            Boeing 747      58840.011691
Name: consumo_kg, dtype: float64

3. Por compañia y tipo de avion el destino más visitado

In [59]:
df_aviones["Destino"].mode()

0    Ginebra
Name: Destino, dtype: object

In [60]:
def mas_visitado(serie):
    return serie.mode()[0] #Di hay varios destinos empatados, escoge el primero que aparezca

In [61]:
df_grouped["Destino"].apply(mas_visitado)

Aircompany  avion      
Airnar      Airbus A320    Los Angeles
            Airbus A380          París
            Boeing 737            Bali
            Boeing 747           Cádiz
FlyQ        Airbus A320        Ginebra
            Airbus A380           Bali
            Boeing 737            Roma
            Boeing 747            Roma
MoldaviAir  Airbus A320          Cádiz
            Airbus A380        Londres
            Boeing 737            Bali
            Boeing 747            Bali
PamPangea   Airbus A320          París
            Airbus A380        Ginebra
            Boeing 737         Ginebra
            Boeing 747       Melbourne
TabarAir    Airbus A320           Roma
            Airbus A380        Ginebra
            Boeing 737      Nueva York
            Boeing 747         Londres
Name: Destino, dtype: object

Vale, ya tenemos las tres cosas que queríamos... por separado. ¿Cómo lo hago a la vez?  
Con `agg` o `aggregate`.

# Agg, aggregate: aplicando funciones y operaciones diferentes por columnas

Muy sencillo, vamos a usar el método `agg` o `aggregate`, al que se le pasa un diccionario:

```python
{
    'nombre_de_la_columna': 'operacion_sobre_esa_columna',
    'nombre_de_la_columna2': 'operacion_sobre_esa_columna',
    ...
    'nombre_de_la_columnan': 'operacion_sobre_esa_columna'
}

In [63]:
agregaciones = {
    "Origen": len,  # count no se reconoce, hay que usar len
    "consumo_kg": "mean",  # Para mean hay que usar la etiqueta "mean"
    "Destino": mas_visitado  # Función personalizada para el destino más visitado
}

In [64]:
df_grouped[["Origen","consumo_kg","Destino"]].agg(agregaciones)

Origen     consumo_kg      Destino
Aircompany avion                                          
Airnar     Airbus A320      37   15233.354249  Los Angeles
           Airbus A380      64  101402.622197        París
           Boeing 737       49   25213.516883         Bali
           Boeing 747       68   87208.773559        Cádiz
FlyQ       Airbus A320      34   12255.748389      Ginebra
           Airbus A380      61  116860.268168         Bali
           Boeing 737       54   20474.087328         Roma
           Boeing 747       67   86157.201917         Roma
MoldaviAir Airbus A320      35    8422.846644        Cádiz
           Airbus A380      74  149316.553108      Londres
           Boeing 737       84   30233.624810         Bali
           Boeing 747       71  133625.459193         Bali
PamPangea  Airbus A320      31    7689.835407        París
           Airbus A380      73  135216.291382      Ginebra
           Boeing 737       66   28712.313147      Ginebra
           Boeing 747       61  127566.722008    Melbourne
TabarAir   Airbus A320      61   15545.639162         Roma
           Airbus A380      71   62575.392039      Ginebra
           Boeing 737       62   12305.056732   Nueva York
           Boeing 747       77   58840.011691      Londres

Por supuesto, podemos usar as_index

In [67]:
df_aviones.groupby(["Aircompany","avion"],as_index=False)[["Origen","consumo_kg","Destino"]].agg(agregaciones)

,Aircompany,avion,Origen,consumo_kg,Destino
0,Airnar,Airbus A320,37,15233.354249,Los Angeles
1,Airnar,Airbus A380,64,101402.622197,París
2,Airnar,Boeing 737,49,25213.516883,Bali
3,Airnar,Boeing 747,68,87208.773559,Cádiz
4,FlyQ,Airbus A320,34,12255.748389,Ginebra
5,FlyQ,Airbus A380,61,116860.268168,Bali
6,FlyQ,Boeing 737,54,20474.087328,Roma
7,FlyQ,Boeing 747,67,86157.201917,Roma
8,MoldaviAir,Airbus A320,35,8422.846644,Cádiz
9,MoldaviAir,Airbus A380,74,149316.553108,Londres


# Groupby: Transform
Vamos a dedicar esta sesión al metodo transform como cierre de las sesiones dedicadas a las agrupaciones hechas con groupby. Pero como tónica habitual de esta unidad, carga datos en nuestro DataFrame guía:

In [68]:
import numpy as  np
import pandas as pd


In [70]:
df_aviones = pd.read_csv("C:/Users/david/Downloads/dataset_inicial_aviones.csv",index_col="Id_vuelo")

Aunque lo vamos a ver en el contexto de las agrupaciones hechas con "groupby", transform es un método que también se aplica a Series y DataFrame sin que haya un groupby de por medio.

Es similar a apply, y sus principales diferencias son:

* Puede aplicar una o varias funciones (sí varias funciones a la vez, introducidas como lista o diccionario)
* Sólo se puede aplicar a una serie (o columna) a la vez (no vale el método de varias columnas)

Supongamos que queremos pasar los Origenes y los Destinos a todo mayúsculas y además generar una abreviatura con las tres primeras letras, en vez de hacerlo en dos pasadas podemos:

In [72]:
def a_mayusculas(valor):
    return valor.upper()
def abrevia(valor):
    return valor[0:3]

In [74]:
df_aviones["Destino"].transform([a_mayusculas,abrevia])

,a_mayusculas,abrevia
Id_vuelo,,
Air_PaGi_10737,GINEBRA,Gin
Fly_BaRo_10737,ROMA,Rom
Tab_GiLo_11380,LOS ANGELES,Los
Mol_PaCi_10737,CINCINNATI,Cin
Tab_CiRo_10747,ROMA,Rom
...,...,...
Tab_LoLo_11320,LONDRES,Lon
Mol_CiLo_10737,LONDRES,Lon
Fly_RoCi_11320,CINCINNATI,Cin


In [75]:
transformaciones={"Des_Mayuscula":a_mayusculas,
                  "Des_Abrev":abrevia}

In [76]:
df_aviones[["Destino","Des_Abrev"]]=df_aviones["Destino"].transform(transformaciones)

In [77]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion,Des_Abrev
Id_vuelo,,,,,,,,
Air_PaGi_10737,Airnar,París,GINEBRA,411,Boeing 737,1028.691900,51,Gin
Fly_BaRo_10737,FlyQ,Bali,ROMA,12738,Boeing 737,33479.132544,1167,Rom
Tab_GiLo_11380,TabarAir,Ginebra,LOS ANGELES,9103,Airbus A380,109439.907200,626,Los
Mol_PaCi_10737,MoldaviAir,París,CINCINNATI,6370,Boeing 737,17027.010000,503,Cin
Tab_CiRo_10747,TabarAir,Cincinnati,ROMA,7480,Boeing 747,86115.744000,518,Rom
...,...,...,...,...,...,...,...,...
Tab_LoLo_11320,TabarAir,Los Angeles,LONDRES,8785,Airbus A320,24766.953120,756,Lon
Mol_CiLo_10737,MoldaviAir,Cincinnati,LONDRES,6284,Boeing 737,16491.729600,497,Lon
Fly_RoCi_11320,FlyQ,Roma,CINCINNATI,7480,Airbus A320,19721.049920,662,Cin


# Transform para agrupaciones
La diferencia más importante respecto a apply o agg es que transform devuelve un serie con tantos elementos como la serie que se le pasa como input (recuerda que transform solo puede manejar un "columna" o serie a la vez). En concreto para los elementos de una misma agrupación devuelve el valor calculado por la función, veámoslo con un ejemplo.

In [ ]:
### Sin transform

In [78]:
df_aviones.groupby(["Aircompany", "avion"])["duracion"].mean()

Aircompany  avion      
Airnar      Airbus A320     486.756757
            Airbus A380     587.921875
            Boeing 737      832.897959
            Boeing 747      567.573529
FlyQ        Airbus A320     407.058824
            Airbus A380     704.803279
            Boeing 737      687.185185
            Boeing 747      577.044776
MoldaviAir  Airbus A320     279.028571
            Airbus A380     895.824324
            Boeing 737     1005.500000
            Boeing 747      877.028169
PamPangea   Airbus A320     260.548387
            Airbus A380     818.356164
            Boeing 737      957.106061
            Boeing 747      843.409836
TabarAir    Airbus A320     494.786885
            Airbus A380     364.014085
            Boeing 737      386.064516
            Boeing 747      370.805195
Name: duracion, dtype: float64

In [79]:
### Con transform
df_aviones.groupby(["Aircompany", "avion"])["duracion"].transform("mean")


Id_vuelo
Air_PaGi_10737     832.897959
Fly_BaRo_10737     687.185185
Tab_GiLo_11380     364.014085
Mol_PaCi_10737    1005.500000
Tab_CiRo_10747     370.805195
                     ...     
Tab_LoLo_11320     494.786885
Mol_CiLo_10737    1005.500000
Fly_RoCi_11320     407.058824
Tab_RoLo_10747     370.805195
Air_PaLo_10737     832.897959
Name: duracion, Length: 1200, dtype: float64

In [80]:
df_aviones["media_duracion"] = df_aviones.groupby(["Aircompany", "avion"])["duracion"].transform('mean')

In [81]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion,Des_Abrev,media_duracion
Id_vuelo,,,,,,,,,
Air_PaGi_10737,Airnar,París,GINEBRA,411,Boeing 737,1028.691900,51,Gin,832.897959
Fly_BaRo_10737,FlyQ,Bali,ROMA,12738,Boeing 737,33479.132544,1167,Rom,687.185185
Tab_GiLo_11380,TabarAir,Ginebra,LOS ANGELES,9103,Airbus A380,109439.907200,626,Los,364.014085
Mol_PaCi_10737,MoldaviAir,París,CINCINNATI,6370,Boeing 737,17027.010000,503,Cin,1005.500000
Tab_CiRo_10747,TabarAir,Cincinnati,ROMA,7480,Boeing 747,86115.744000,518,Rom,370.805195
...,...,...,...,...,...,...,...,...,...
Tab_LoLo_11320,TabarAir,Los Angeles,LONDRES,8785,Airbus A320,24766.953120,756,Lon,494.786885
Mol_CiLo_10737,MoldaviAir,Cincinnati,LONDRES,6284,Boeing 737,16491.729600,497,Lon,1005.500000
Fly_RoCi_11320,FlyQ,Roma,CINCINNATI,7480,Airbus A320,19721.049920,662,Cin,407.058824


Y ahora podemos hacer cálculos directos que de otra forma serían más complejos de conseguir. Vamos a crearnos otra columna que recoja para cada vuelo el porcentaje sobre la media de su duración lo que nos permitirá luego por ejemplo hacer control de vuelos que se nos desvían mucho o poco de la media.

Tal como lo tenemos es fácil hacer esa columna

In [82]:
df_aviones["desviacion_duracion"]=df_aviones["duracion"]/df_aviones["media_duracion"]

In [83]:
df_aviones

,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion,Des_Abrev,media_duracion,desviacion_duracion
Id_vuelo,,,,,,,,,,
Air_PaGi_10737,Airnar,París,GINEBRA,411,Boeing 737,1028.691900,51,Gin,832.897959,0.061232
Fly_BaRo_10737,FlyQ,Bali,ROMA,12738,Boeing 737,33479.132544,1167,Rom,687.185185,1.698232
Tab_GiLo_11380,TabarAir,Ginebra,LOS ANGELES,9103,Airbus A380,109439.907200,626,Los,364.014085,1.719714
Mol_PaCi_10737,MoldaviAir,París,CINCINNATI,6370,Boeing 737,17027.010000,503,Cin,1005.500000,0.500249
Tab_CiRo_10747,TabarAir,Cincinnati,ROMA,7480,Boeing 747,86115.744000,518,Rom,370.805195,1.396960
...,...,...,...,...,...,...,...,...,...,...
Tab_LoLo_11320,TabarAir,Los Angeles,LONDRES,8785,Airbus A320,24766.953120,756,Lon,494.786885,1.527931
Mol_CiLo_10737,MoldaviAir,Cincinnati,LONDRES,6284,Boeing 737,16491.729600,497,Lon,1005.500000,0.494281
Fly_RoCi_11320,FlyQ,Roma,CINCINNATI,7480,Airbus A320,19721.049920,662,Cin,407.058824,1.626301
